# Drehwinkel einer Punktwolke

Gegeben sind für $k = 1, 2$ die Mengen $$P_k = \{ (x^{(k)}_1, y^{(k)}_1), \ldots, (x^{(k)}_N, y^{(k)}_N)\}$$
von Punkten $(x^{(k)}_i, y^{(k)}_i) \in \mathbb{R}^2$, welche als Matrizen gespeichert werden, d.h. $P_1, P_2 \in \mathbb{R}^{N \times 2}$. Zusätzlich
ist die Drehungsmatrix

$$
    M(\alpha) = \begin{pmatrix} 
    \cos{(\alpha)} & -\sin({\alpha}) \\ 
    \sin{(\alpha)} & \cos{(\alpha)}
    \end{pmatrix}
$$

gegeben, wobei $0 \leq \alpha \leq 2\pi$ der Drehwinkel ist.

Gesucht ist der Drehwinkel $\alpha^* = \arg \min_{\alpha} f(\alpha)$, wobei
$$
f(\alpha) 
= \frac{1}{N} \mathrm{Tr}{\left( G G^\top\right)} 
= \frac{1}{N} \sum_{i = 1}^{N} g_{i1}^2 + g_{i2}^2.
$$
mit $G = P_2 - P_1 M(\alpha)^\top$. Dazu kann z.B. `scipy.optimize.minimize_scalar` verwendet werden, welches einige Ableitungsfreie Verfahren enthält. Die Laufzeit zum Berechnen von $\alpha^*$ hängt somit entscheidend davon ab, wie schnell wir $f$ auswerten können.
Wir nehmen an, dass das obige Problem als laufzeitkritischer Teil eines größeren Algorithmus innerhalb einer Schleife ggf. 100 000
mal gelöst werden muss.

In [ ]:
%load_ext cython

In [ ]:
%load_ext line_profiler

In [ ]:
# Gegebene Punkte:

import numpy as np

# generate some data for demonstration purposes
# points in each point cloud are ordered by correspondence
num_points = 500

distance = np.random.rand(num_points) * 3
radii = np.random.rand(num_points) * 2*np.pi
pc1 = distance[:, None] * np.stack([np.cos(radii), np.sin(radii)], axis=1)

distance = np.random.rand(num_points) * 3
radii = np.random.rand(num_points) * 2*np.pi
pc2 = distance[:, None] * np.stack([np.cos(radii), np.sin(radii)], axis=1)

Gegeben ist folgende Implementierung der Zielfunktion. Was ist hier ineffizient? Was könnte beschleunigt werden?

In [ ]:
from math import sin, cos

# Zielfunktion
def score(alpha, pc1, pc2):
    rot_matrix = np.array([
        [cos(alpha), -sin(alpha)],
        [sin(alpha), cos(alpha)]
    ])
    pc1_rotated = pc1 @ rot_matrix.T

    sum_of_squares = np.sum((pc2 - pc1_rotated)**2, axis=1)
    mse = np.mean(sum_of_squares)

    return mse

In [ ]:
score(0.25, pc1, pc2)

In [ ]:
%lprun -f score score(0.5, pc1, pc2)

# Cython

In [ ]:
%%cython -a -f -c=-O3 -c=-march=native -c=-Wno-deprecated-declarations -c=-Wno-#warnings

from math import cos, sin
cimport numpy as np
import numpy as np
from cython cimport wraparound, boundscheck

@wraparound(False)
@boundscheck(False)
def score2(double alpha, double[:, ::1] pc1, double[:, ::1] pc2):
    cdef int i, N = pc1.shape[0]
    cdef double[:, ::1] pc1rot
    cdef double mse, diff1, diff2
    
    rot_matrixT = np.array([[cos(alpha), sin(alpha)],[-sin(alpha), cos(alpha)]])
    pc1rot = pc1 @ rot_matrixT
    
    #sum_of_squares = np.sum((pc2 - pc1_rotated)**2, axis=1)
    #mse = np.mean(sum_of_squares)

    for i in range(N):
        diff1 = pc2[i,0] - pc1rot[i,0]
        diff2 = pc2[i,1] - pc1rot[i,1]
        mse  += diff1*diff1 + diff2*diff2
    
    return mse / N

In [ ]:
score2(0.25, pc1, pc2)

In [ ]:
%timeit score(0.25, pc1, pc2)

In [ ]:
%timeit score2(0.25, pc1, pc2)

In [ ]:
%%cython -a -f -c=-O3 -c=-march=native -c=-Wno-deprecated-declarations -c=-Wno-#warnings

from libc.math cimport cos, sin
cimport numpy as np
import numpy as np
from cython cimport wraparound, boundscheck

@wraparound(False)
@boundscheck(False)
cpdef double score3(double alpha, double[:, ::1] pc1, double[:, ::1] pc2):
    cdef int i
    cdef int N = pc1.shape[0]
    cdef double[:, ::1] rot_matrixT
    cdef double diff1
    cdef double diff2
    cdef double mse = 0.0
    rot_matrixT = np.array([[cos(alpha), sin(alpha)],[-sin(alpha), cos(alpha)]])

    for i in range(N):
        diff1 = pc2[i,0] - (pc1[i,0]*rot_matrixT[0,0] + pc1[i,1]*rot_matrixT[1,0])
        diff2 = pc2[i,1] - (pc1[i,0]*rot_matrixT[0,1] + pc1[i,1]*rot_matrixT[1,1])
        mse  += diff1*diff1 + diff2*diff2
    return mse / N

In [ ]:
score3(0.5, pc1, pc2)

In [ ]:
%timeit score3(0.5, pc1, pc2)

In [ ]:
%%cython -a -f -c=-O3 -c=-march=native -c=-Wno-deprecated-declarations -c=-Wno-#warnings

from libc.math cimport cos, sin
cimport numpy as np
import numpy as np
from cython cimport wraparound, boundscheck

@wraparound(False)
@boundscheck(False)
cpdef double score4(double alpha, double[:, ::1] pc1, double[:, ::1] pc2):
    cdef int i
    cdef int N = pc1.shape[0]
    cdef double diff1 = 0.0
    cdef double diff2 = 0.0
    cdef double   mse = 0.0
    cdef double  rmT00 = cos(alpha)
    cdef double  rmT01 = sin(alpha)
    cdef double  rmT10 = -rmT01
    cdef double  rmT11 = rmT00

    for i in range(N):
        diff1 = pc2[i,0] - (pc1[i,0]*rmT00 + pc1[i,1]*rmT10)
        diff2 = pc2[i,1] - (pc1[i,0]*rmT01 + pc1[i,1]*rmT11)
        mse  += diff1*diff1 + diff2*diff2
    return mse / N

In [ ]:
%timeit score4(0.5, pc1, pc2)